<font color=red> 
# Part 1

### Submit a link to your Notebook on your Github repository. (10 marks)

In [1]:
import numpy as np 
import pandas as pd 
from bs4 import BeautifulSoup, Comment
import urllib

### Get the HTML data

In [2]:
link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
f = urllib.request.urlopen(link)
info_raw=f.read()

In [3]:
soup = BeautifulSoup(info_raw,'lxml')

### Retreave only the Table

In [4]:
table = soup.table

### Import the table form HTML into a panda dataframe

In [5]:
df = pd.read_html(str(table))[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


### Remove the rows in which Borough is not assigned

In [6]:
df = df[df['Borough'] != 'Not assigned']
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


### Make Neighborhood seperate by commas

In [7]:
df['Neighborhood']=df['Neighborhood'].str.replace('/',',')
df.head()   


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


### In case there are rows in wich the Neighborhood is Not assigned 
(just in case, there are actually no row like that)

In [8]:

df=df[~df['Neighborhood'].str.contains('Not assigned')]

### Size of the dataframe

In [9]:
df.shape

(103, 3)

<font color=red> 
# Part 2

### Once you are able to create the above dataframe, submit a link to the new Notebook on your Github repository. (2 marks)

In [10]:
# pip install pgeocode

### Dear reviewer: Due to technical difficaulties I was adivised by the course technical stuff to use pgeocode package.
### Please note that this package is acceptable for this asignment. 

In [11]:
import pgeocode
nomi = pgeocode.Nominatim('CA')

In [12]:
df.set_index('Postal code', inplace = True)

In [13]:
for postal_code, row in df.iterrows():
    g = nomi.query_postal_code(postal_code)
    df.at[postal_code, 'Latitude'] = g.latitude
    df.at[postal_code, 'Longitude'] = g.longitude
    

In [14]:
df

,Borough,Neighborhood,Latitude,Longitude
Postal code,,,,
M3A,North York,Parkwoods,43.7545,-79.3300
M4A,North York,Victoria Village,43.7276,-79.3148
M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
M6A,North York,"Lawrence Manor , Lawrence Heights",43.7223,-79.4504
M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...
M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.6518,-79.5076
M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.3830
M7Y,East Toronto,Business reply mail Processing CentrE,43.7804,-79.2505


<font color=red> 
# Part 3

### Once you are happy with your analysis, submit a link to the new Notebook on your Github repository. (3 marks)

In [15]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import requests # library to handle requests

### Get coordinates of Toronto

In [16]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [17]:
# Remove rows that does not ontains coordinates informaiton
df = df[df['Longitude'].notnull()]

### Get only neighborhoods in Toronto

In [18]:
toronto_data = df[df['Borough'].str.contains('Toronto')].reset_index()
toronto_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M4E,East Toronto,The Beaches,43.6784,-79.2941


### create map of Toronto using latitude and longitude values

In [19]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### This is the credential to Foursquare

In [20]:
CLIENT_ID = 'XBTPHG4SLH5BGUF44S4RODQQUXEFTZ1LSGNP5L3OCCFWVJBC' # your Foursquare ID
CLIENT_SECRET = 'U524JMFYTFZFHUWAWNO5IEIP3QXJCLQOCZNADOB3NRHHWRBT'
 # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

### Creat a function to return the nearby venues

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    radius=500
    LIMIT=100
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get the nearby venues

In [22]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Regent Park , Harbourfront
Queen's Park , Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
The Danforth West , Riverdale
Toronto Dominion Centre , Design Exchange
Brockton , Parkdale Village , Exhibition Place
India Bazaar , The Beaches West
Commerce Court , Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park , The Junction South
North Toronto West
The Annex , North Midtown , Yorkville
Parkdale , Roncesvalles
Davisville
University of Toronto , Harbord
Runnymede , Swansea
Moore Park , Summerhill East
Kensington Market , Chinatown , Grange Park
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Roseda

In [23]:
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park , Harbourfront",43.6555,-79.3626,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park , Harbourfront",43.6555,-79.3626,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park , Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park , Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,"Regent Park , Harbourfront",43.6555,-79.3626,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
...,...,...,...,...,...,...,...
1527,Business reply mail Processing CentrE,43.7804,-79.2505,Winners,43.782743,-79.252767,Department Store
1528,Business reply mail Processing CentrE,43.7804,-79.2505,Milestones,43.778060,-79.255321,Restaurant
1529,Business reply mail Processing CentrE,43.7804,-79.2505,Hwy 401 at McCowan,43.780220,-79.255161,Intersection
1530,Business reply mail Processing CentrE,43.7804,-79.2505,Ultimate Martial Arts,43.777320,-79.248554,Martial Arts Dojo


### Make dummies of the venues 

In [24]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
toronto_onehot= toronto_onehot.set_index(['Neighborhood'], drop=True)
toronto_onehot=toronto_onehot.reset_index()
# toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()


,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park , Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Group together the neighborhoods

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.010989,0.021978,0.000000,0.000000,0.000000,0.0,0.010989,0.0,...,0.0,0.0,0.010989,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.010989
1,"Brockton , Parkdale Village , Exhibition Place",0.025641,0.000000,0.025641,0.000000,0.025641,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.062500
3,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.017544,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.017544,0.0,0.017544
4,Central Bay Street,0.000000,0.000000,0.000000,0.016949,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.016949,0.016949,0.0,0.0,0.000000,0.0,0.000000


In [26]:
toronto_onehot.shape

(1532, 212)

In [27]:
toronto_grouped.shape

(38, 212)

### A function to sort the venues in descending order.

In [28]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create a new dataframe and display the top 10 venues for each neighborhood

In [29]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Hotel,Cocktail Bar,Beer Bar,Japanese Restaurant,Seafood Restaurant,Bakery,Restaurant,Cheese Shop
1,"Brockton , Parkdale Village , Exhibition Place",Coffee Shop,Café,Thrift / Vintage Store,Breakfast Spot,Gift Shop,Italian Restaurant,Gym,Brewery,Hawaiian Restaurant,Sandwich Place
2,Business reply mail Processing CentrE,Restaurant,Coffee Shop,Yoga Studio,Bank,Martial Arts Dojo,Salad Place,Breakfast Spot,Bookstore,Sushi Restaurant,Japanese Restaurant
3,"CN Tower , King and Spadina , Railway Lands , ...",Coffee Shop,Restaurant,Bar,Café,Italian Restaurant,Speakeasy,Bakery,Bank,Gym / Fitness Center,Park
4,Central Bay Street,Coffee Shop,Café,Middle Eastern Restaurant,Italian Restaurant,Restaurant,Breakfast Spot,Sandwich Place,Bubble Tea Shop,Shoe Store,Spa


### Cluster simmilar Neighborhoods together

In [30]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [31]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 0, 4, 4, 4, 2])

In [32]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged =toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.6555,-79.3626,4.0,Coffee Shop,Breakfast Spot,Yoga Studio,Beer Store,Health Food Store,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.6641,-79.3889,4.0,Sushi Restaurant,Coffee Shop,Gym,Italian Restaurant,Park,Creperie,Distribution Center,Poke Place,Ethiopian Restaurant,Diner
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,4.0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Restaurant,Cosmetics Shop,Italian Restaurant,Japanese Restaurant,Hotel,Bubble Tea Shop
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,4.0,Coffee Shop,Café,Seafood Restaurant,Italian Restaurant,American Restaurant,Cocktail Bar,Gastropub,Clothing Store,Bakery,Theater
4,M4E,East Toronto,The Beaches,43.6784,-79.2941,4.0,Pub,Health Food Store,Trail,Bakery,Gastropub,Eastern European Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop


In [33]:
#  Delete rows that does not asigned to any cluster
toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)

In [34]:
# Change datatype of the cluster into integer
toronto_merged['Cluster Labels']= toronto_merged['Cluster Labels'].astype(int)


### visualize the resulting clusters

In [35]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)  
map_clusters

### As you can see, there are 5 groups of neighbourhoods seperated by different coulors.